# Exercice entretien 
#### Leo Bierent (05/05/2020)

### Instructions :
You need to flatten the attached tree (tree_to_convert.txt) into a set of strategies. 

A strategy is a combination of 

● A strategy definition: a sequence of conditions of the form feature = value or feature
!= value, separated with “and” operators only.

● A leaf value
The syntax is given by [strategy definition]:[leaf_value].

For instance, leaf 4 translates into the following strategy : device_type!=pc & browser!=7 & browser=8 : 0.000881108


In [1]:
import pandas as pd
import numpy as np

On va definir une classe pour chaque noeud. Ainsi, on pourra acceder à toutes les informations concernant le noeud et ses dépendances très simplement. 

Les noeuds sont de la forme : 

● 19:leaf=0.000594593 (Noeud final)

● 11:[size=300x600] yes=18,no=17 (Noeud intermediaire)

In [2]:
class node:
    
    def __init__(self, str_row):
        
        # Le numero du noeud est juste avant le premier ':'
        self.num = int(str_row.split()[0].split(':')[0])
        # Je vais mettre ces valeurs à nan pour aviter les erreurs de comparaison par la suite. (Peut etre remplacé par des try / Except)
        self.valeur = np.nan
        self.condition = np.nan
        self.right_child = np.nan
        self.left_child = np.nan
        
        # Il faut savoir si cette feuille correspond à un bout de branche.
        # J'ai choisi de retenir cet indicateur ('[') de feuille finale car detecter la présence de 'leaf' n'aurait pas été robuste 
        # à un nom de variable contenant 'leaf' 
        
        if  '[' not in str_row :
            # On prend la partie après '='
            self.valeur = float(str_row.split('=')[1])
            
        else :
            # Permet d'isoler la partie entre []
            self.condition = str_row[str_row.find("[")+1:str_row.find("]")]
            
            #Pour les enfants, on prend la partie de droite (yes, no)
            childs = str_row.split()[1]
            self.right_child = int(childs.split(',')[0].split('=')[1])
            self.left_child = int(childs.split(',')[1].split('=')[1])
            
    def __str__(self):
        return str(self.num)
    
    # Cette fonction permet de retrouver le parent du noeud et de savoir si la condition du parent est vraie ou fausse. 
    
    def get_parent(self,arbre):
        self.parent = np.nan
        self.parent_bool = np.nan
        
        for noeud in arbre :
            if noeud.right_child == self.num:
                self.parent = noeud
                self.parent_bool = True 
                
            if noeud.left_child == self.num:
                self.parent = noeud
                self.parent_bool = False
            


On va egalement créer une classe simple pour notre arbre, qui initialise bien les parents des noeuds

In [3]:
class tree:
    def __init__(self,node_list):
        self.nodes = node_list
        
        for node in self.nodes:
            node.get_parent(self.nodes)



Il faut parfois prendre la négation de la condition du parent, qui peut etre de la forme A U B 
On utilise alors la loi de Morgan pour prendre la negation. 
Pour l'instant le programme ne gère que des égalités et des unions, on peut facilement augmenter ses capacités en ajoutant l'operateur souhaité et na negation dans le dictionnaire operation_inv

In [4]:
operations_inv = {
    '=' : ' != ',
    '||or||' : ' & ',
    ## Ajouter des operateurs si besoin ! 
    }

# Fonction qui remplace tous les operateurs par leur negation si la condition parente est fausse !

def morgan(condition,parent_bool):

    if not parent_bool :
        for op in operations_inv:

            condition = condition.replace(op,operations_inv[op])
            
    return condition

    


On va maintenant pouvoir créer une fonction qui remonte l'arbre depuis les branches finales jusqu'au noeud zero (Supposons que ne noeud zero soit toujours noté zero, sinon, on peut changer la condition d'arret)


In [5]:
#Fonction recursive, qui concatenate les conditions l'une après l'autre, en ajoutant & entre elles 
# et en prenant en compte les negations. 
#Possible de la faire en iteratif pour de plus grands arbres 

def flat_(noeud, flat = ''):
    # On ajoute à flat la condition du parent corrigée si besoin
    flat = flat+' & ' + morgan(noeud.parent.condition,noeud.parent_bool)
    #condition d'arret
    if noeud.parent.num == 0 :
        return flat[len(' & '):]
    
    return flat_(noeud.parent,flat)


On obtient donc une liste de toutes les conditions pour arriver jusqu'au noeud final. 
Dans les consignes, il est précisé qu'il ne doit pas y avoir d'operateur 'ou' dans la strategie. 
Alors, on va distribuer les strategies de façon à faire disparaitre les 'ou'. 


A & (B U C) : cst   

devient 


A & B : cst     
A & C : cst


In [6]:
def split_operator(cond):
    # On peut changer cette variable si il y a changement de syntaxe 
    op_or = '||or||'
    
    if len(cond.split(op_or))>1:
        # On va partitionner, ce qui va nous separer notre str au niveau de la première occurence du 'ou'
        part = cond.partition(op_or)
        op_and = operations_inv[op_or]

        # On obtient deux nouvelles conditions avec un 'ou' distibué en plus
        left = op_and.join(part[0].split(op_and)[:-1]+[part[2]])
        right = op_and.join([part[0]]+ part[2].split(op_and)[1:])
        
        # On fait encore cette operation sur les nouvelles conditions, au cas où il resterait des 'ou'
        return [split_operator(left),split_operator(right)]
    
    return cond


A cause de la recursivite de la fonction precedente, on obtient des resultats sous forme de liste de liste
on va créer une petite fonction pour rendre ces resultats flat

In [7]:
def flatten(l): 
    return flatten(l[0]) + (flatten(l[1:]) if len(l) > 1 else []) if type(l) is list else [l]


Il ne reste plus qu'à appliquer les fonctions precedentes aux feuilles en bout de branche. 

In [8]:
def get_result(arbre):
    result = []
    for noeud in arbre.nodes : 
        
        # On ne prend que les feuilles avec des valeurs
        if  ~np.isnan(noeud.valeur):
            
            cond_list = split_operator(flat_(noeud))
            cond_list = flatten(cond_list)
            
            # On met en forme comme dans le feuille de consignes
            cond_list = ['{} : {}'.format(cond,noeud.valeur) for cond in cond_list ]
            result = result + cond_list
            
    return result
      

On n'a plus qu'à créer la fonction main qui ouvre le fichier, créé l'arbre, calcule les strategies et les ecrit dans le fichier txt. 

In [76]:
def main(text_file):
    # On ouvre le dataset
    data = pd.read_csv(text_file, sep="\n", header=None,names = ['node'])
    
    #Création de l'arbre
    arbre = tree([node(noeud) for noeud in data.node.values])
    
    #Calcul des strategies
    result = get_result(arbre)
    
    # Ecriture des resultats
    strat_file = 'strategies.txt'
    #print('Ecriture des {} strategies dans {} : \n\n'.format(len(result),strat_file))

    file = open(strat_file,'w') 
    for strategie in result : 
        file.write(strategie+'\n')
        #print(strategie)


    file.close() 
    return result



On obtient ici 20 strategies (pour 11 noeuds finaux)

In [108]:
main('tree_to_convert.txt')


['language=2 & browser=8 & os_family=5 & browser=7 : 0.000559453',
 'language=2 & browser=8 & os_family=5 & device_type=pc : 0.000559453',
 'language != 2 & browser=8 & os_family=5 & browser=7 : 0.000594593',
 'language != 2 & browser=8 & os_family=5 & device_type=pc : 0.000594593',
 'size=300x600 & browser != 8 & os_family=5 & browser=7 : 0.000597397',
 'size=300x600 & browser != 8 & os_family=5 & device_type=pc : 0.000597397',
 'size != 300x600 & browser != 8 & os_family=5 & browser=7 : 0.00063461',
 'size != 300x600 & browser != 8 & os_family=5 & device_type=pc : 0.00063461',
 'browser=5 & os_family != 5 & browser=7 : 0.000625534',
 'browser=5 & os_family != 5 & device_type=pc : 0.000625534',
 'browser=8 & os_family != 5 & browser=7 : 0.000625534',
 'browser=8 & os_family != 5 & device_type=pc : 0.000625534',
 'position=2 & browser != 8 & browser != 5 & os_family != 5 & browser=7 : 0.00066727',
 'position=2 & browser != 8 & browser != 5 & os_family != 5 & device_type=pc : 0.00066727

In [106]:
import random
import unittest

class treeTest(unittest.TestCase):
    
    def test_len(self):
        result_test = main('tree_to_convert.txt')
        self.assertEqual(len(result_test),20)
        
    def test_or(self):
        result_test = main('tree_to_convert.txt')
        self.assertNotIn('||or||', result_test)
    
    def test_len_big_depth(self):
        
        result_test= main('Test/tree_big_depth.txt')
        self.assertEqual(len(result_test),24)
        
    def test_or_big_depth(self):
        result_test= main('Test/tree_big_depth.txt')
        self.assertNotIn('||or||', result_test)
    
    def test_len_small_depth(self):
        
        result_test= main('Test/tree_small_depth.txt')
        self.assertEqual(len(result_test),6)
        
    def test_or_small_depth(self):
        result_test= main('Test/tree_small_depth.txt')
        self.assertNotIn('||or||', result_test)
        
    def test_len_variable_change(self):
        
        result_test= main('Test/tree_variable_change.txt')
        self.assertEqual(len(result_test),20)
        
    def test_or_variable_change(self):
        result_test= main('Test/tree_variable_change.txt')
        self.assertNotIn('||or||', result_test)
      
    # Test pour plus de conditions ||or|| ce qui a pour effet une explosion du nombre de strategies. 
    def test_len_variable_change(self):
        
        result_test= main('Test/tree_lot_OR.txt')
        self.assertEqual(len(result_test),64)
    
    # Controle qu'il ne reste plus d''||or||
    def test_or_variable_change(self):
        result_test= main('Test/tree_lot_OR.txt')
        self.assertNotIn('||or||', result_test)


In [107]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

........
----------------------------------------------------------------------
Ran 8 tests in 0.056s

OK
